In [21]:
import pandas as pd

In [64]:
from tf.app import use
A = use('etcbc/bhsa', hoist=globals())

A.load(['g_prs', 'g_nme', 'g_pfm', 'g_vbs', 'g_vbe'])

**Locating corpus resources ...**

Name,# of nodes,# slots/node,% coverage
book,39,10938.21,100
chapter,929,459.19,100
lex,9230,46.22,100
verse,23213,18.38,100
half_verse,45179,9.44,100
sentence,63717,6.70,100
sentence_atom,64514,6.61,100
clause,88131,4.84,100
clause_atom,90704,4.70,100
phrase,253203,1.68,100


True

In [65]:
valid_suff_chars = {'H', 'J', 'K', 'M', 'N', 'W'}

In [66]:
lexemes_with_suff = set()
for w in F.otype.s('word'):
    
    is_perfect = F.vt.v(w) == 'perf' and F.ps.v(w) == 'p3' and F.nu.v(w) == 'pl'
    is_imperfect = F.vt.v(w) == 'impf' and F.ps.v(w) in {'p2', 'p3'} and F.nu.v(w) == 'pl' and F.gn.v(w) == 'm'
    is_imperative = F.vt.v(w) == 'impv' and F.nu.v(w) == 'pl' and F.gn.v(w) == 'm'
    
    if is_perfect or is_imperfect or is_imperative:
        suff = ''.join([char for char in F.g_prs.v(w) if char in valid_suff_chars])
        if suff:
            lexemes_with_suff.add(F.lex.v(w))
            
len(lexemes_with_suff)

212

In [67]:
vbe_set = set()

info_dict = {}

for w in F.otype.s('word'):
    
    is_perfect = F.vt.v(w) == 'perf' and F.ps.v(w) == 'p3' and F.nu.v(w) == 'pl'
    is_imperfect = F.vt.v(w) in {'impf', 'wayq'} and F.ps.v(w) in {'p2', 'p3'} and F.nu.v(w) == 'pl' and F.gn.v(w) == 'm'
    is_imperative = F.vt.v(w) == 'impv' and F.nu.v(w) == 'pl' and F.gn.v(w) == 'm'
    lang = F.language.v(w)
    if is_perfect or is_imperfect or is_imperative and lang == 'Hebrew':
        suff = ''.join([char for char in F.g_prs.v(w) if char in valid_suff_chars])
        #print(F.lex_utf8.v(w))
        vbe = F.g_vbe.v(w)

        vbe_set.add(vbe)
        
        lex = F.lex.v(w)
        g_cons = F.g_cons.v(w)
        has_suffix = int(len(suff) > 0)
        bo, ch, ve = T.sectionFromNode(w)
        if F.qere.v(w) != None:
            continue
        if vbe in {'[','[H','[J', '[W.>', '[WJ'}:
            print(w, bo, ch, ve, vbe, F.g_cons.v(w), suff, F.qere.v(w))
        if lex in lexemes_with_suff:
            if vbe not in {'[U', '[W', '[W.', '[W.-',}:
                #print(w, bo, ch, ve, vbe, F.g_cons.v(w), suff, F.qere.v(w))
                continue
            
            g_cons_no_suff = F.g_cons.v(w).rstrip(suff)
            suff_has_w = int('W' in suff)
            verb_ends_on_w = int(g_cons_no_suff[-1] == 'W')
            if not g_cons_no_suff.endswith('WN'):
                info_dict[w] = [w, bo, ch, ve, lex, F.lex_utf8.v(w), g_cons, g_cons_no_suff, suff, 
                    F.vt.v(w), F.vs.v(w), F.gn.v(w), F.nu.v(w), F.ps.v(w), verb_ends_on_w, has_suffix, suff_has_w] 

118926 Joshua 10 24 [W.> HLKW>  None
221017 Isaiah 28 12 [W.> >BW>  None
239977 Jeremiah 10 5 [W.> JNFW>  None


In [68]:
vbe_set

{'[',
 '[H',
 '[J',
 '[ON',
 '[OW',
 '[OWN',
 '[TEM',
 '[U',
 '[UN',
 '[UN.',
 '[W',
 '[W.',
 '[W.-',
 '[W.>',
 '[W.N',
 '[W.N.',
 '[WJ',
 '[WN'}

In [69]:
len(info_dict)

7090

In [70]:
dat = pd.DataFrame(info_dict).T

dat.columns = ['tf_id', 'book', 'chapter', 'verse', 'lex', 'lex_hebrew', 'g_cons', 'g_cons_no_suff', 'suff', 'vt', 'vs', 
               'gn', 'nu', 'ps', 'verb_ends_on_w', 'has_suffix', 'w_in_suffix']
dat.head(20)

,tf_id,book,chapter,verse,lex,lex_hebrew,g_cons,g_cons_no_suff,suff,vt,vs,gn,nu,ps,verb_ends_on_w,has_suffix,w_in_suffix
424,424,Genesis,1,22,ML>[,מלא,ML>W,ML>W,,impv,qal,m,pl,p2,1,0,0
564,564,Genesis,1,28,ML>[,מלא,ML>W,ML>W,,impv,qal,m,pl,p2,1,0,0
569,569,Genesis,1,28,KBC[,כבשׁ,KBCH,KBC,H,impv,qal,m,pl,p2,0,1,0
675,675,Genesis,2,1,KLH[,כלה,JKLW,JKLW,,wayq,pual,m,pl,p3,1,0,0
1192,1192,Genesis,3,1,>KL[,אכל,T>KLW,T>KLW,,impf,qal,m,pl,p2,1,0,0
1224,1224,Genesis,3,3,>KL[,אכל,T>KLW,T>KLW,,impf,qal,m,pl,p2,1,0,0
1303,1303,Genesis,3,7,JD<[,ידע,JD<W,JD<W,,wayq,qal,m,pl,p3,1,0,0
1312,1312,Genesis,3,7,<FH[,עשׂה,J<FW,J<FW,,wayq,qal,m,pl,p3,1,0,0
1316,1316,Genesis,3,8,CM<[,שׁמע,JCM<W,JCM<W,,wayq,qal,m,pl,p3,1,0,0
1752,1752,Genesis,4,5,NPL[,נפל,JPLW,JPLW,,wayq,qal,m,pl,p3,1,0,0


In [71]:
dat.to_csv('plural_verbs_mt.csv', sep='\t')

In [72]:
pd.crosstab(dat.verb_ends_on_w, dat.has_suffix)

has_suffix,0,1
verb_ends_on_w,,
0,0,332
1,6358,400


In [73]:
pd.crosstab(dat.verb_ends_on_w, dat.w_in_suffix)

w_in_suffix,0,1
verb_ends_on_w,,
0,80,252
1,6758,0


In [74]:
from tf.app import use
A = use('dt-ucph/sp', hoist=globals())

A.load(['g_prs', 'g_nme', 'g_pfm', 'g_vbs', 'g_vbe'])

**Locating corpus resources ...**

The requested data is not available offline
	~/text-fabric-data/github/dt-ucph/sp/tf/3.3.1 not found


Name,# of nodes,# slots/node,% coverage
book,5,79878.20,100
chapter,187,2135.78,100
verse,5841,68.38,100
word,114890,3.48,100
sign,399391,1.00,100


True

In [75]:
lexemes_with_suff

{'<BD[',
 '<BR[',
 '<CQ[',
 '<FH[',
 '<LH[',
 '<MM[',
 '<NH[',
 '<RK[',
 '<WD[',
 '<WT[',
 '<YB=[',
 '<YB[',
 '<ZB[',
 '<ZR[',
 '>CR=[',
 '>HB[',
 '>KL[',
 '>MR[',
 '>PP[',
 '>RH[',
 '>SR[',
 '>XZ[',
 'B<L[',
 'B<T[',
 'BHL[',
 'BL<[',
 'BNH[',
 'BQC[',
 'BQQ[',
 'BRK[',
 'BTQ[',
 'BW>[',
 'BXN[',
 'BZZ[',
 'C>L[',
 'CBH[',
 'CBR[',
 'CBX[',
 'CDD[',
 'CJT[',
 'CKL[',
 'CKR[',
 'CKX[',
 'CLK[',
 'CLL[',
 'CLM[',
 'CLX[',
 'CM<[',
 'CMC[',
 'CMD[',
 'CMV[',
 'CNH[',
 'CPV[',
 'CQH[',
 'CRT[',
 'CSS[',
 'CTH[',
 'CVP[',
 'CVX[',
 'CWB[',
 'CXR[',
 'DBQ[',
 'DJG[',
 'DK>[',
 'DLQ[',
 'DMH[',
 'DPQ[',
 'DQR[',
 'DRC[',
 'DRK[',
 'F<R==[',
 'FJM[',
 'FMX[',
 'FN>[',
 'FRP[',
 'FVM[',
 'FVN[',
 'FWF[',
 'G>L[',
 'GDL[',
 'GML[',
 'GNB[',
 'GRC[',
 'HDP[',
 'HLL[',
 'HLM[',
 'HRG[',
 'J<L[',
 'J<Y[',
 'JBL[',
 'JC<[',
 'JD<[',
 'JDH[',
 'JR>[',
 'JRC[',
 'JRD[',
 'JRH=[',
 'JRH[',
 'JSR[',
 'JY>[',
 'JYT[',
 'K<S[',
 'KBC[',
 'KBD[',
 'KLH[',
 'KLM[',
 'KRH[',
 'KSH[',
 'KTR[',
 'KWL[',
 'LJY

In [76]:
vbe_set = set()

info_dict = {}

for w in F.otype.s('word'):
    
    is_perfect = F.vt.v(w) == 'perf' and F.ps.v(w) == 'p3' and F.nu.v(w) == 'pl'
    is_imperfect = F.vt.v(w) in {'impf', 'wayq'} and F.ps.v(w) in {'p2', 'p3'} and F.nu.v(w) == 'pl' and F.gn.v(w) == 'm'
    is_imperative = F.vt.v(w) == 'impv' and F.nu.v(w) == 'pl' and F.gn.v(w) == 'm'
    lang = F.language.v(w)
    if is_perfect or is_imperfect or is_imperative and lang == 'Hebrew':
        suff = ''.join([char for char in F.g_prs.v(w) if char in valid_suff_chars])
        #print(F.lex_utf8.v(w))
        vbe = F.g_vbe.v(w)
        vbe_set.add(vbe)
        
        lex = F.lex.v(w)
        g_cons = F.g_cons.v(w)
        has_suffix = int(len(suff) > 0)
        bo, ch, ve = T.sectionFromNode(w)
        if lex in lexemes_with_suff:
            if vbe == '[WN':
                continue
            
            g_cons_no_suff = F.g_cons.v(w).rstrip(suff)
            suff_has_w = int('W' in suff)
            verb_ends_on_w = int(g_cons_no_suff[-1] == 'W')
            if not g_cons_no_suff.endswith('WN'):
                info_dict[w] = [w, bo, ch, ve, lex, F.lex_utf8.v(w), g_cons, g_cons_no_suff, suff, 
                    F.vt.v(w), '-', F.gn.v(w), F.nu.v(w), F.ps.v(w), verb_ends_on_w, has_suffix, suff_has_w]

In [77]:
dat = pd.DataFrame(info_dict).T

dat.columns = ['tf_id', 'book', 'chapter', 'verse', 'lex', 'lex_hebrew', 'g_cons', 'g_cons_no_suff', 'suff', 'vt', 'vs', 
               'gn', 'nu', 'ps', 'verb_ends_on_w', 'has_suffix', 'w_in_suffix']
dat.head(20)

,tf_id,book,chapter,verse,lex,lex_hebrew,g_cons,g_cons_no_suff,suff,vt,vs,gn,nu,ps,verb_ends_on_w,has_suffix,w_in_suffix
405847,405847,Genesis,1,22,ML>[,מלא[,ML>W,ML>W,,impv,-,m,pl,p2,1,0,0
405986,405986,Genesis,1,28,ML>[,מלא[,ML>W,ML>W,,impv,-,m,pl,p2,1,0,0
405991,405991,Genesis,1,28,KBC[,כבשׁ[,KBCWH,KBCW,H,perf,-,unknown,pl,p3,1,1,0
406098,406098,Genesis,2,1,KLH[,כלה[,JKLW,JKLW,,impf,-,m,pl,p3,1,0,0
406616,406616,Genesis,3,1,>KL[,אכל[,T>KLW,T>KLW,,impf,-,m,pl,p2,1,0,0
406650,406650,Genesis,3,3,>KL[,אכל[,T>KLW,T>KLW,,impf,-,m,pl,p2,1,0,0
406722,406722,Genesis,3,6,>KL[,אכל[,J>KLW,J>KLW,,impf,-,m,pl,p3,1,0,0
406728,406728,Genesis,3,7,JD<[,ידע[,JD<W,JD<W,,impf,-,m,pl,p3,1,0,0
406737,406737,Genesis,3,7,<FH[,עשׂה[,J<FW,J<FW,,impf,-,m,pl,p3,1,0,0
406741,406741,Genesis,3,8,CM<[,שׁמע[,JCM<W,JCM<W,,impf,-,m,pl,p3,1,0,0


In [78]:
dat.to_csv('plural_verbs_sp.csv', sep='\t')

In [79]:
dat.shape

(753, 17)

In [80]:
pd.crosstab(dat.verb_ends_on_w, dat.has_suffix)

has_suffix,0,1
verb_ends_on_w,,
0,0,7
1,703,43
